In [1]:
from cosapp.drivers import RungeKutta
from cosapp.recorders import DataFrameRecorder
from cosapp.drivers import NonLinearSolver, RunOnce, MonteCarlo, RunSingleCase
from cosapp.utils.distributions import Normal
import sys
sys.path.append('./model')
from Earth import Earth
import json
import numpy as np

earth = Earth("earth")

l = 2 #Rocket's length on the plot
dt = 0.05 #Time-step


from cosapp.utils.distributions import Normal
from cosapp.drivers import MonteCarlo
from cosapp.recorders import DataFrameRecorder

In [2]:

#Initial conditions and constants

rocketNum = 2622
l = 0.855
angz = -np.deg2rad(85)
expCoef = False #True for experimental drag coefficient, false for analytical (possibly innacurate)
Wind = False #True for wind effects, false for no wind
Lift = False #True for lift effects (trajecto does not consider them), false for no lift

init = {
    'Traj.r' : np.array([-(l/2)*np.sin(angz), 0., (l/2)*np.cos(angz)]),
    'Para.DynPar.r1' : np.array([0., 0., l/2]),
    'Para.DynPar.r2' : np.array([0., 0., l/2]),
    'Rocket.Kin.ar' : np.array([0, angz, 0]),
    'Rocket.CG' : l/2,
    
    'Rocket.Mass.m' : 2.0783,
    'Rocket.Mass.m0' : 2.0783,
    'Rocket.Mass.Dm' : 0.076,
    'Rocket.Mass.lastEngineTime' : 1.,
    'Rocket.Mass.I0_geom' : 1*np.array([1., 100., 100.]),

    'Rocket.Kin.v' : np.array([0,0,0]),
    'Rocket.Kin.av' : np.zeros(3),
    'Para.DynPar.v1' : np.array([0,0,0]),
    'Para.DynPar.v2' : np.array([0,0,0]),
    'Traj.ParaDepStatus': False,

    'Rocket.Aero.Coefs.ln' : 0.159,
    'Rocket.Aero.Coefs.d' : 0.08,
    'Rocket.Aero.Coefs.NFins' : 4,
    'Rocket.Aero.Coefs.s' : 0.13,
    'Rocket.Aero.Coefs.Xt' : 0.09,
    'Rocket.Aero.Coefs.Cr' : 0.145,
    'Rocket.Aero.Coefs.Ct' : 0.09,
    'Rocket.Aero.Coefs.tf' : 0.002,
    # 'Rocket.Aero.Coefs.delta' : 0.,

    'Rocket.Aero.Coefs.TypeCd' : expCoef,
    'Wind.wind_on' : Wind,
    'Rocket.Aero.Aeroforces.isLift': Lift,
    'Rocket.Aero.Coefs.Cd_exp': 0.6,

    'Traj.parachute_deploy_method' : 0,
    'Traj.parachute_deploy_timer' : 0.,

}

In [3]:
def run_analysis(syst, draws=10, linear=True):
    syst.drivers.clear()  # Remove all drivers on the System

    # runonce = syst.add_driver(RunOnce("runonce"))
    driver = syst.add_driver(RungeKutta(order=4, dt=dt))
    # driver.add_recorder(DataFrameRecorder(includes=['Rocket.Kin.ar', 'Traj.r', 'Rocket.Kin.v', 'initrot', 'Rocket.Thrust.inclinaison']), period=0.1)
    driver.add_child(NonLinearSolver('solver', factor=1.0))
    driver.time_interval = (0, 70)

    # Define a simulation scenario
    driver.set_scenario(
        init = init,
        stop = 'Para.DynPar.r2[2] < -1'
    )

    syst.run_drivers()

    print("MONTECARLO")
    # Montecarlo
    syst.drivers.clear()
    montecarlo = syst.add_driver(MonteCarlo('mc'))
    montecarlo.add_recorder(DataFrameRecorder(includes=['Para.DynPar.r2', 'Para.DynPar.v2', 'Rocket.Aero.Coefs.delta']))
    # montecarlo.add_child(runonce)
    montecarlo.add_child(driver)
    montecarlo.draws = draws
    # montecarlo.linear = linear

    # parameters for uncertainties in the data
    cant_attr = syst.Rocket.Aero.Coefs.inwards.get_details('delta')
    s_attr = syst.Rocket.Aero.Coefs.inwards.get_details('s')
    tf_attr = syst.Rocket.Aero.Coefs.inwards.get_details('tf')
    
    # Set the distribution around the current value
    cant_attr.distribution = Normal(best=.01, worst=-.01)
    s_attr.distribution = Normal(best=.01, worst=-.01)
    tf_attr.distribution = Normal(best=.001, worst=-.001)

    montecarlo.add_random_variable('Rocket.Aero.Coefs.delta')
    montecarlo.add_random_variable('Rocket.Aero.Coefs.s')
    montecarlo.add_random_variable('Rocket.Aero.Coefs.tf')

    # Computation
    syst.run_drivers()

    return montecarlo.recorder.export_data()

In [4]:

results = run_analysis(earth, draws=50, linear=False)

___PARACHUTE DEPLOYMENT___


Rocket X Coordinate at Deployment:  30.87794567410323 m
Rocket Y Coordinate at Deployment:  0.0 m
Rocket Z Coordinate at Deployment:  169.80336380164607 m


___PARACHUTE DEPLOYMENT___


Parachute fully deployed !
MONTECARLO
___PARACHUTE DEPLOYMENT___


Rocket X Coordinate at Deployment:  30.87794567410323 m
Rocket Y Coordinate at Deployment:  0.0 m
Rocket Z Coordinate at Deployment:  169.8033638016461 m


___PARACHUTE DEPLOYMENT___


Parachute fully deployed !
___PARACHUTE DEPLOYMENT___


Rocket X Coordinate at Deployment:  30.87794567410323 m
Rocket Y Coordinate at Deployment:  0.0 m
Rocket Z Coordinate at Deployment:  169.8033638016461 m


___PARACHUTE DEPLOYMENT___


Parachute fully deployed !
___PARACHUTE DEPLOYMENT___


Rocket X Coordinate at Deployment:  30.87794567410323 m
Rocket Y Coordinate at Deployment:  0.0 m
Rocket Z Coordinate at Deployment:  169.8033638016461 m


___PARACHUTE DEPLOYMENT___


Parachute fully deployed !
___PARACHUTE DEPLOYMENT_

In [5]:
results

,Section,Status,Error code,Reference,Para.DynPar.r2,Para.DynPar.v2,Rocket.Aero.Coefs.delta
0,,,0,0,"[38.85438334502149, 4.5299774981559205e-33, -1...","[6.641400073819258e-07, 2.855267570860113e-40,...",-9.656293e-19
1,,,0,1,"[35.19643301285781, 10.292761692209105, -0.999...","[4.946527258387128e-07, 3.824846738569515e-07,...",6.507796e-03
2,,,0,2,"[35.196433013014705, -10.292761692778578, -0.9...","[4.946527264945277e-07, -3.824846714659121e-07...",-6.507796e-03
3,,,0,3,"[38.00540121188944, -5.212619509836143, -1.000...","[6.242250271632009e-07, -1.9949866485682844e-0...",-3.074383e-03
4,,,0,4,"[29.22146295322445, 14.673633053999348, -0.999...","[2.3344840161654473e-07, 5.034026541069057e-07...",1.109912e-02
5,,,0,5,"[38.00540121222089, 5.212619509587429, -1.0000...","[6.2422498016217e-07, 1.9949866050825012e-07, ...",3.074383e-03
6,,,0,6,"[29.22146295322445, -14.673633053999348, -0.99...","[2.3344840161654473e-07, -5.034026541069057e-0...",-1.109912e-02
7,,,0,7,"[32.758174958512775, -12.658384351889866, -0.9...","[3.855203054270177e-07, -4.5686247077971203e-0...",-8.559610e-03
8,,,0,8,"[36.88708741063139, 7.78452516507399, -1.00000...","[5.721691055504737e-07, 2.945912768491264e-07,...",4.715946e-03
9,,,0,9,"[23.73134732940869, 15.593262129436734, -1.000...","[1.5713045964371268e-08, 4.773364836918266e-07...",1.480192e-02


In [6]:

with open('result.txt', 'a') as fp:
    fp.write(results.to_csv())

In [7]:
traj = np.asarray(results['Para.DynPar.r2'].tolist())
traj

array([[ 3.88543833e+01,  4.52997750e-33, -1.00000000e+00],
       [ 3.51964330e+01,  1.02927617e+01, -1.00000000e+00],
       [ 3.51964330e+01, -1.02927617e+01, -1.00000000e+00]])

In [6]:

import plotly.graph_objs as go
from plotly.subplots import make_subplots

traj = np.asarray(results['Para.DynPar.r2'].tolist())

# Create the figure object
fig = make_subplots(rows=1, cols=1)
fig.layout.title = "Probability"
fig.layout.yaxis.title = 'Y Position'

fig.add_trace(
    go.Scatter(
        x=traj[:,0],
        y=traj[:,1],
        mode = 'markers'
    ),
    row = 1,
    col = 1,
)
fig.get_subplot(1, 1).xaxis.title = "X Position"

fig.show()